In [1]:
import mlflow
import os
import requests
import pandas as pd
from minio import Minio
from io import BytesIO

# ======================
# LightGBM 학습 (불균형 보정 + 조기종료) - MLflow 연동
# ======================
# 테스트용 데이터 임의 생성 (입력)
import numpy as np
import pandas as pd
import os
import pickle
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

import socket

In [2]:
mlflow_server_ip = socket.gethostbyname("mlflow-server")
mlflow.set_tracking_uri(f"http://{mlflow_server_ip}:5000")
mlflow.set_experiment("test_classifier")

<Experiment: artifact_location='s3://mlflow/3', creation_time=1761710958662, experiment_id='3', last_update_time=1761710958662, lifecycle_stage='active', name='test_classifier', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [3]:
# MinIO 클라이언트 초기화
client = Minio(
    "mlflow-minio:9000",  # MinIO 서버 주소
    access_key="minio",
    secret_key="minio123",
    secure=False  # HTTPS를 사용하지 않는 경우 (프로덕션 환경에서는 True로 설정)
)

In [4]:
# 모든 년도 파일 리스트
bucket_name = "prepro"
years = ['2020', '2021', '2022', '2023', '2024', '2025']
dfs = []

for year in years:
    object_name = f"restaurant_{year}.csv"
    response = client.get_object(bucket_name, object_name)
    data_stream = BytesIO(response.read())
    df_temp = pd.read_csv(data_stream)
    dfs.append(df_temp)
    response.close()
    response.release_conn()

# 모든 DataFrame 합치기
df = pd.concat(dfs, ignore_index=True)

df = df.copy()
df = df.drop(columns=['num'])

le = LabelEncoder()
label_data = le.fit_transform(df['영업상태명'])

df['label'] = label_data
# 주소지 컬럼 인코딩
import re

# 번지 숫자 추출 함수
def extract_lot_number(lot_str):
    """번지에서 숫자를 추출합니다 (예: '3', '11', '169-3' -> 3, 11, 169)"""
    if pd.isna(lot_str) or lot_str == '' or lot_str == 'nan':
        return 0
    # 첫 번째 숫자 추출
    match = re.search(r'\d+', str(lot_str))
    return int(match.group()) if match else 0

# 번지 컬럼 숫자로 변환
df['번지_숫자'] = df['번지'].apply(extract_lot_number)

# 시도, 시군구, 읍면동 LabelEncoder 인코딩
cols = ['시도', '시군구', '읍면동', '구분']
address_encoders = {}

for col in cols:
    if col in df.columns:
        le = LabelEncoder()
        # NaN 값을 '기타'로 채우기
        df[col] = df[col].fillna('기타')
        df[f'{col}_encoded'] = le.fit_transform(df[col])
        address_encoders[col] = le

# 도로명은 카디널리티가 너무 높아 제외하거나, 빈도 기반으로 필터링
# 필요시 아래 코드 활성화
# from collections import Counter
# road_freq = Counter(df['도로명'].fillna(''))
# popular_roads = {road for road, count in road_freq.items() if count > 10}
# df['도로명_인기'] = df['도로명'].apply(lambda x: x if x in popular_roads else '기타')
# le_road = LabelEncoder()
# df['도로명_인기_encoded'] = le_road.fit_transform(df['도로명_인기'])

print("주소지 인코딩 완료")



unused_cols = ['개방서비스명', '인허가일자', '폐업일자', '영업상태명', '소재지', 
               'label', '시도', '시군구', '읍면동', '번지', '도로명', 
               '시설명', '남성종사자수', '여성종사자수']  # 시설명, 종사자수도 제외
features = [col for col in df.columns if col not in unused_cols]

X = df[features]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
cat_features = []
for col in X_train.columns:
    if "encoded" in col or X_train[col].dtype == "object":
        cat_features.append(col)
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
X_train = X_train[numeric_cols].astype('float64')
X_test = X_test[numeric_cols].astype('float64')


주소지 인코딩 완료


In [5]:
import joblib

# 학습된 모델 불러오기
model = joblib.load('./model.pkl')

# 간단 예측: 테스트셋의 일부 샘플 예측해 보기
sample_X = X_test.iloc[:5]
preds = model.predict(sample_X)
print("샘플 예측 결과:", preds)

샘플 예측 결과: [0 1 1 1 0]


# 인코더

In [6]:
# 주소지 컬럼 인코딩 (기존 코드)
import re
from sklearn.preprocessing import LabelEncoder
import joblib
import os

# 번지 숫자 추출 함수
def extract_lot_number(lot_str):
    """번지에서 숫자를 추출합니다"""
    if pd.isna(lot_str) or lot_str == '' or lot_str == 'nan':
        return 0
    match = re.search(r'\d+', str(lot_str))
    return int(match.group()) if match else 0

# 번지 컬럼 숫자로 변환
df['번지_숫자'] = df['번지'].apply(extract_lot_number)

# 시도, 시군구, 읍면동 LabelEncoder 인코딩
cols = ['시도', '시군구', '읍면동', '구분']
address_encoders = {}

for col in cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = df[col].fillna('기타')
        df[f'{col}_encoded'] = le.fit_transform(df[col])
        address_encoders[col] = le

print("주소지 인코딩 완료")

# ✅ 인코더 저장 추가 (여기에 추가!)
ENCODER_SAVE_PATH = './encoders.pkl'  # 또는 모델과 같은 위치
joblib.dump(address_encoders, ENCODER_SAVE_PATH)
print(f"✅ 인코더 저장 완료: {ENCODER_SAVE_PATH}")

# MLflow에도 저장하고 싶다면 (선택사항)
# mlflow.log_artifact(ENCODER_SAVE_PATH, "encoders")

주소지 인코딩 완료
✅ 인코더 저장 완료: ./encoders.pkl


In [7]:
# 모델 저장 위치와 동일하게
MODEL_DIR = './'  # 또는 './models/'
ENCODER_PATH = os.path.join(MODEL_DIR, 'encoders.pkl')
joblib.dump(address_encoders, ENCODER_PATH)

['./encoders.pkl']